# Setup Environment

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import yaml

# Data Preparing

In [ ]:
def create_data_yaml(train_path, val_path, num_classes, class_names, output_path='data.yaml'):
    """
    Creates a 'data.yaml' file to be used for training a model, specifying paths for the 
    training and validation datasets, number of classes, and class names.

    Args:
    - train_path (str): Path to the training dataset images.
    - val_path (str): Path to the validation dataset images.
    - num_classes (int): Number of classes in the dataset.
    - class_names (list): List of class names.
    - output_path (str, optional): Path where the 'data.yaml' file will be saved. Default is 'data.yaml'.

    Returns:
    - None: This function writes to a file and does not return any value.
    """
    with open(output_path, 'w') as f:
        f.write(f"train: {train_path}\n")
        f.write(f"val: {val_path}\n")
        f.write(f"nc: {num_classes}\n")
        f.write(f"names: {class_names}\n")
    print(f"data.yaml created at {output_path}")


train_path = "License-Plates.v1i.yolov8/train/images" 
val_path = "License-Plates.v1i.yolov8/valid/images"    
num_classes = int("1") 
class_names = "license_plate".split(',')

class_names = [name.strip() for name in class_names]

create_data_yaml(train_path, val_path, num_classes, class_names, output_path='data.yaml')

yaml_path = 'data.yaml'

with open(yaml_path, 'r') as f:
    data = f.read() 
    print(data)


# Model Fine-Tuning

In [ ]:
model = YOLO('/content/license_plate_detector.pt')

model.train(data=data, epochs=5, imgsz=100, batch=32)

model.save('fine_tuned_linecne_plate.pt')


# Model Evaluate

In [ ]:
results = model.val(data='/content/vehicle-and-license-plate-3/data.yaml', imgsz=640)

results_dict = results.results_dict
print(f"Precision: {results_dict['metrics/precision(B)']:.4f}")
print(f"Recall: {results_dict['metrics/recall(B)']:.4f}")
print(f"mAP@0.5: {results_dict['metrics/mAP50(B)']:.4f}")
print(f"mAP@0.5:0.95: {results_dict['metrics/mAP50-95(B)']:.4f}")
print(f"F1 Score: {2 * (results_dict['metrics/precision(B)'] * results_dict['metrics/recall(B)']) / (results_dict['metrics/precision(B)'] + results_dict['metrics/recall(B)']):.4f}")